In [1]:
import mxnet as mx
from mxnet import gluon
from mxnet import nd
import numpy as np

In [2]:
mx.ndarray.Convolution

<function ndarray.Convolution>

In [10]:
# 输入输出数据格式是 batch x channel x height x width，这里batch和channel都是1
# 权重格式是 output_channels x in_channels x height x width
w = nd.arange(4).reshape((1, 1, 2, 2))
b = nd.array([1])
data = nd.arange(9).reshape([1, 1, 3, 3])

In [12]:
out = nd.Convolution(data, w, b, kernel=weight.shape[2:], num_filter=w.shape[1], stride=(2, 2), pad=(1, 1))
print('input:', data, '\n\nweight:', w, '\n\nbias:', b, '\n\noutput:', out)

input: 
[[[[ 0.  1.  2.]
   [ 3.  4.  5.]
   [ 6.  7.  8.]]]]
<NDArray 1x1x3x3 @cpu(0)> 

weight: 
[[[[ 0.  1.]
   [ 2.  3.]]]]
<NDArray 1x1x2x2 @cpu(0)> 

bias: 
[ 1.]
<NDArray 1 @cpu(0)> 

output: 
[[[[  1.   9.]
   [ 22.  44.]]]]
<NDArray 1x1x2x2 @cpu(0)>


In [13]:
w = nd.arange(8).reshape((1,2,2,2))
data = nd.arange(18).reshape((1,2,3,3))

out = nd.Convolution(data, w, b, kernel=w.shape[2:], num_filter=w.shape[0])

print('input:', data, '\n\nweight:', w, '\n\nbias:', b, '\n\noutput:', out)

input: 
[[[[  0.   1.   2.]
   [  3.   4.   5.]
   [  6.   7.   8.]]

  [[  9.  10.  11.]
   [ 12.  13.  14.]
   [ 15.  16.  17.]]]]
<NDArray 1x2x3x3 @cpu(0)> 

weight: 
[[[[ 0.  1.]
   [ 2.  3.]]

  [[ 4.  5.]
   [ 6.  7.]]]]
<NDArray 1x2x2x2 @cpu(0)> 

bias: 
[ 1.]
<NDArray 1 @cpu(0)> 

output: 
[[[[ 269.  297.]
   [ 353.  381.]]]]
<NDArray 1x1x2x2 @cpu(0)>


In [14]:
w = nd.arange(16).reshape((2,2,2,2))
data = nd.arange(18).reshape((1,2,3,3))
b = nd.array([1,2])

out = nd.Convolution(data, w, b, kernel=w.shape[2:], num_filter=w.shape[0])

print('input:', data, '\n\nweight:', w, '\n\nbias:', b, '\n\noutput:', out)

input: 
[[[[  0.   1.   2.]
   [  3.   4.   5.]
   [  6.   7.   8.]]

  [[  9.  10.  11.]
   [ 12.  13.  14.]
   [ 15.  16.  17.]]]]
<NDArray 1x2x3x3 @cpu(0)> 

weight: 
[[[[  0.   1.]
   [  2.   3.]]

  [[  4.   5.]
   [  6.   7.]]]


 [[[  8.   9.]
   [ 10.  11.]]

  [[ 12.  13.]
   [ 14.  15.]]]]
<NDArray 2x2x2x2 @cpu(0)> 

bias: 
[ 1.  2.]
<NDArray 2 @cpu(0)> 

output: 
[[[[  269.   297.]
   [  353.   381.]]

  [[  686.   778.]
   [  962.  1054.]]]]
<NDArray 1x2x2x2 @cpu(0)>


In [15]:
# Pooling

In [16]:
data = nd.arange(18).reshape((1, 2, 3, 3))

max_pooling = nd.Pooling(data=data, pool_type='max', kernel=(2, 2), stride=(2, 2), pad=(1, 1))
avg_pooling = nd.Pooling(data=data, pool_type='avg', kernel=(3, 3))

In [17]:
print(max_pooling)
print(avg_pooling)


[[[[  0.   2.]
   [  6.   8.]]

  [[  9.  11.]
   [ 15.  17.]]]]
<NDArray 1x2x2x2 @cpu(0)>

[[[[  4.]]

  [[ 13.]]]]
<NDArray 1x2x1x1 @cpu(0)>


In [18]:
import sys
sys.path.append('.')
from utils import load_data_fashion_mnist

batch_size = 256
train_data, test_data = load_data_fashion_mnist(batch_size)

ImportError: cannot import name 'load_data_fashion_mnist'

In [19]:
import mxnet as mx

try:
    ctx = mx.gpu()
    _ = nd.zeros((1,), ctx=ctx)
except:
    ctx = mx.cpu()
ctx

cpu(0)

In [23]:
# LeNet
weight_scale = .01
# output channels = 20, kernel = (5, 5)
W1 = nd.random.normal(shape=(20, 1, 5, 5), scale=weight_scale, ctx=ctx)
b1 = nd.random.normal(shape=W1.shape[0], ctx=ctx)
# output channel = 50, kernel = (3, 3)
W2 = nd.random.normal(shape=(50, 20, 3, 3), scale=weight_scale, ctx=ctx)
b2 = nd.random.normal(shape=W2.shape[0], ctx=ctx)
# output dim = 128
W3 = nd.random.normal(shape=(1250, 128), scale=weight_scale, ctx=ctx)
b3 = nd.zeros(shape=W3.shape[1], ctx=ctx)
# output dim = 10
W4 = nd.random.normal(shape=(W3.shape[1], 10), scale=weight_scale, ctx=ctx)
b4 = nd.zeros(shape=W4.shape[1], ctx=ctx)

params = [W1, b1, W2, b2, W3, b3, W4, b4]
for param in params:
    param.attach_grad()

In [25]:
# conv + activation + pooling
def net(X, verbose=False):
    X = X.as_in_context(W1.context)
    # conv1
    h1_conv = nd.Convolution(data=X, weight=W1, bias=b1, kernel=W1[2:], num_filter=W1.shape[0])
    h1_activation = nd.relu(h1_conv)
    h1 = nd.Pooling(data=h1_activation, pool_type='max', kernel=(2, 2), stride=(2, 2))
    # conv2
    h2_conv = nd.Convolution(data=h1, weight=W2, bias=b2, kernel=W2[2:], num_filter=W2.shape[0])
    h2_activation = nd.relu(h2_conv)
    h2 = nd.Pooling(data=h2_activation, pool_type='max', kernel=[2, 2], stride=[2, 2])
    # fc1
    h2 = nd.flatten(h2)
    h3_linear = nd.dot(h2, W3) + b3
    h3 = nd.relu(h3_linear)
    # fc2
    h4_linear = hd.dot(h3, W4) + b4
    if verbose:
        print('1st conv block:', h1.shape)
        print('2nd conv block:', h2.shape)
        print('1st dense:', h3.shape)
        print('2nd dense:', h4_linear.shape)
        print('output:', h4_linear)
    return h4_linear

In [26]:
for data, _ in train_data:
    net(data, verbose=True)
    break

NameError: name 'train_data' is not defined

In [29]:
# Training
from mxnet import autograd as autograd
from utils import SGD, accuracy, evaluate_accuracy
from mxnet import gluon

softmax_cross_entropy_loss = gluon.loss.SoftmaxCrossEntropyLoss()
learning_rate = .2
for epoch in range(5):
    train_loss = 0.
    train_acc = 0.
    for data, label in train_data:
        label = label.as_in_context(ctx)
        with autograd.record():
            output = net(data)
            loss = softmax_cross_entropy_loss(output, label)
        loss.backward()
        SGD(params, learning_rate/batch_size)
        
        train_loss += nd.mean(loss).asscalar()
        train_acc += accuracy(output, label)
    
    test_acc = evaluate_accuracy(test_data, net, ctx)
    print("Epoch %d. Loss: %f, Train acc %f, Test acc %f" % (epoch, train_loss/len(train_data), train_acc/len(train_data), test_acc))

ImportError: cannot import name 'SGD'